In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as col#, when
import csv

# spark = SparkSession(sc)
# sqlContext = SQLContext(sc)

In [2]:
tktsFile = 'Parking_Violations_Issued_2015_simplified.csv'
geo_file =  'cscl.csv'

In [3]:
sc = SparkContext()
sqlContext = SQLContext(sc)
directory = 'hdfs:///data/share/bdm/nyc_parking_violations/'

files = []

In [4]:
def parseCSV(idx,part):
    if idx == 0:
        next(part)
    for p in csv.reader(part):
        if p[23].isalpha() and p[24]=='' and p[21]=='' and p[23]=='':
            continue
        if '-' in p[23]:
            yield(p[23].split('-')[0],p[23].split('-')[1],p[24],p[21],p[4][-4:]) 
        else:
            yield(p[23],"",p[24],p[21],p[4][-4:])

In [5]:
rows = sc.textFile(tktsFile).mapPartitionsWithIndex(parseCSV)
#rows.collect(5)
df = sqlContext.createDataFrame(rows,('House Number','HN Compound','Street Name', 'County','Year'))

In [6]:
df.count()

5000

In [7]:
df.show(10)

+------------+-----------+-----------+------+----+
|House Number|HN Compound|Street Name|County|Year|
+------------+-----------+-----------+------+----+
|         158|           |    8th Ave|    NY|2015|
|          10|           |  E 29th St|    NY|2015|
|          46|           | William St|    NY|2014|
|           S|           |  Remsen St|     K|2015|
|           W|           |   161st St|     Q|2014|
|         115|           | W 172nd St|    BX|2015|
|         350|           |  W 58th St|    NY|2015|
|        1965|           |   Broadway|    NY|2014|
|        8027|           |Jamaica Ave|     Q|2015|
|          82|         46|   135th St|     Q|2015|
+------------+-----------+-----------+------+----+
only showing top 10 rows



In [8]:
def parseCL(idx,part):
    if idx == 0:
        next(part)
    for p in csv.reader(part):
        LL_HN = p[0]
        LL_HNC = ''
        LH_HN = p[0]
        LH_HNC = ''
        if '-' in p[0] and '-' in p[1]:#need and statement otherwise may run into case with nulls
            LL_HN = p[0].split('-')[0]
            LL_HNC = p[0].split('-')[1]
            LH_HN = p[1].split('-')[0]
            LH_HNC = p[1].split('-')[1]
        RL_HN = p[4]
        RL_HNC = ''
        RH_HN = p[5]
        RH_HNC = ''
        if '-' in p[4] and '-' in p[5]:
            RL_HN = p[4].split('-')[0]
            RL_HNC = p[4].split('-')[1]
            RH_HN = p[5].split('-')[0]
            RH_HNC = p[5].split('-')[1]
            
        
        yield(p[3], p[28], p[10], p[13], LL_HN, LL_HNC, LH_HN, LH_HNC, RL_HN, RL_HNC, RH_HN, RH_HNC  )

In [9]:
rdd2 = sc.textFile('geo_2015.csv').mapPartitionsWithIndex(parseCL)
df2  = sqlContext.createDataFrame(rdd2,('ID','Full Street','st label','borocode','LL_HN','LL_HNC','LH_HN','LH_HNC','RL_HN','RL_HNC','RH_HN','RH_HNC'))
df2.show(10)
df2.count()

+-------+--------------------+--------------------+--------+------+------+------+------+-----+------+-----+------+
|     ID|         Full Street|            st label|borocode| LL_HN|LL_HNC| LH_HN|LH_HNC|RL_HN|RL_HNC|RH_HN|RH_HNC|
+-------+--------------------+--------------------+--------+------+------+------+------+-----+------+-----+------+
|       |MITSUBISHI WILD W...|MITSUBISHI WILD W...|       2|164809|      |164809|      |     |      |     |      |
|215-027|              28 AVE|               28 AV|       4|  6110|      |  6110|      |  215|   000|  215|   026|
|    399|     SCHERMERHORN ST|     SCHERMERHORN ST|       3|145494|      |145494|      |  316|      |  360|      |
|   5699|       ARLINGTON AVE|        ARLINGTON AV|       2| 61140|      | 61140|      | 5602|      | 5698|      |
|120-011|         QUEENS BLVD|         QUEENS BLVD|       4| 12438|      | 12438|      |    0|      |    0|      |
|    555|       ARTHUR KIL RD|      ARTHUR KILL RD|       5| 15432|      | 15432

999

In [10]:
county = {
    'BROOKLYN':('BK','K','KING','KINGS'),
    'MANHATTAN':('MAN','MH','MN','NEWY','NEW Y','NY'),
    'BRONX':('BRONX','BX'), #MATCH ON SUBSTRING
    'QUEENS': ('Q','QN','QNS','QU','QUEEN'),
    'STATEN ISLAND': ('R','RICHMOND')
}

# ~55million total, 35M matched

In [11]:
cond1 = df['Street Name'] == df2['Full Street']
cond2 = df['Street Name'] == df2['st label']
cond3 = df['County'] == df2['borocode']

joined = df2.join(df, (cond1|cond2) & cond3, 'inner')
left = joined.filter((joined['House Number'] %2 == 1) & joined['HN Compound'].isNull())
right = joined.filter((joined['House Number'] %2 == 0) & joined['HN Compound'].isNull())

left_withc = joined.filter((joined['House Number'] %2 == 1) & joined['HN Compound'].isNotNull())
right_withc = joined.filter((joined['House Number'] %2 == 0) & joined['HN Compound'].isNotNull())



In [12]:

print(left.count())
print(right.count())
print(left_withc.count())
print(right_withc.count())

0
0
0
0
